## GOAL  
Enron was an American energy company who went bankrupt in early 2000 due to financial irregularities. During investigation, number of persons were found guilty and convicted. Goal of this project is to identify employees(Person of Interest-POI) who may have committed fraud using given financial and email exchange data by using machine learning. 

All code here is taken from project code file poi_id.py and explained in following section.

In [1]:
# loading the dataset
import sys
import pickle
import pprint as pp
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Store to my_dataset for easy export below.
my_dataset = data_dict

### SUMMARY OF DATA

Our dataset is a mapping of finance and email exchange data to person's name. We have 'poi' flag as an independent feature, which can be learnt as dependent on other features. We would learn a binomial classifier with 'poi' as predicted variable by other features.

In [2]:
print "TOTAL NUMBER OF PERSONS: ", len(my_dataset)

TOTAL NUMBER OF PERSONS:  146


In [3]:
print "TOTAL NUMBER OF FEATURES: ", len(my_dataset[my_dataset.keys()[1]])

TOTAL NUMBER OF FEATURES:  21


In [4]:
print "LIST OF FEATURES:"
pp.pprint(my_dataset[my_dataset.keys()[1]].keys())

LIST OF FEATURES:
['salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'exercised_stock_options',
 'bonus',
 'restricted_stock',
 'shared_receipt_with_poi',
 'restricted_stock_deferred',
 'total_stock_value',
 'expenses',
 'loan_advances',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'poi',
 'director_fees',
 'deferred_income',
 'long_term_incentive',
 'email_address',
 'from_poi_to_this_person']


In [5]:
all_poi = 0
num_missing_values = {}
for name in my_dataset:
    if my_dataset[name]['poi'] == True:
        all_poi += 1;
    for feature in my_dataset[name]:
        if my_dataset[name][feature] == "NaN":
            if not feature in num_missing_values:
                num_missing_values[feature] = 1
            else:
                num_missing_values[feature] += 1

print "TOTAL NUMBER OF PERSON OF INTEREST: ", all_poi
num_missing_values = {}
for name in my_dataset:
    for feature in my_dataset[name]:
        if my_dataset[name][feature] == "NaN":
            if not feature in num_missing_values:
                num_missing_values[feature] = 1
            else:
                num_missing_values[feature] += 1
print "COUNTS OF MISSING ENTRIES IN DIFFERENT FEATURES: "
pp.pprint(num_missing_values)

TOTAL NUMBER OF PERSON OF INTEREST:  18
COUNTS OF MISSING ENTRIES IN DIFFERENT FEATURES: 
{'bonus': 64,
 'deferral_payments': 107,
 'deferred_income': 97,
 'director_fees': 129,
 'email_address': 35,
 'exercised_stock_options': 44,
 'expenses': 51,
 'from_messages': 60,
 'from_poi_to_this_person': 60,
 'from_this_person_to_poi': 60,
 'loan_advances': 142,
 'long_term_incentive': 80,
 'other': 53,
 'restricted_stock': 36,
 'restricted_stock_deferred': 128,
 'salary': 51,
 'shared_receipt_with_poi': 60,
 'to_messages': 60,
 'total_payments': 21,
 'total_stock_value': 20}


### OUTLIER DETECTION

On manual inspection of data, we found that there are following persons for whom data is not valid so we would delete them from dataset.
- LOCKHART EUGENE E  
All feature values are undefined for this person  


- TOTAL  
- THE TRAVEL AGENCY IN THE PARK  
Above 2 are are not valid persons

In [6]:
pp.pprint(my_dataset['LOCKHART EUGENE E'])

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 'NaN',
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 'NaN',
 'total_stock_value': 'NaN'}


In [7]:
# Delete the outliers
del my_dataset['LOCKHART EUGENE E']
del my_dataset['TOTAL']
del my_dataset['THE TRAVEL AGENCY IN THE PARK']

### NEW FEATURE CREATION

We inspected list of features to find financial features of interest as below:

In [8]:
features_list = ['poi', 'salary', 'deferral_payments', 'total_payments', 'long_term_incentive',
                 'loan_advances', 'bonus', 'restricted_stock', 'restricted_stock_deferred',
                 'total_stock_value', 'exercised_stock_options',
                 'deferred_income', 'expenses', 'director_fees']
pp.pprint(features_list)

['poi',
 'salary',
 'deferral_payments',
 'total_payments',
 'long_term_incentive',
 'loan_advances',
 'bonus',
 'restricted_stock',
 'restricted_stock_deferred',
 'total_stock_value',
 'exercised_stock_options',
 'deferred_income',
 'expenses',
 'director_fees']


For email specific features, we see that count of number of emails exchanges with poi could be low even for a poi person if total emails exchanges for that person is low.  
To mitigate this problem, We create 2 new features
- fraction_from_poi  =  from_poi_to_this_person/to_messages
- fraction_to_poi = from_this_person_to_poi/from_messages

In [9]:
# creating new features
def computeFraction(poi_messages, all_messages):
    """ computes fraction of poi with give messages type and total of that message type """
    fraction = 0.
    if all_messages != "NaN":
        fraction = float(poi_messages)/float(all_messages)
    else:
        fraction = 0
    return fraction

for name in data_dict:
    from_poi_to_this_person = my_dataset[name]["from_poi_to_this_person"]
    to_messages = my_dataset[name]["to_messages"]
    my_dataset[name]["fraction_from_poi"] = computeFraction(from_poi_to_this_person, to_messages)

    from_this_person_to_poi = my_dataset[name]["from_this_person_to_poi"]
    from_messages = my_dataset[name]["from_messages"]
    my_dataset[name]["fraction_to_poi"] = computeFraction(from_this_person_to_poi, from_messages)
features_list.append("fraction_from_poi")
features_list.append("fraction_to_poi")

### LEARNING, TUNING AND VALIDATION WITH FEATURE SELECTION

Performance of an algorithm depends upon its parameters e.g. performance of KNN algorithm would depend upon number of nearest neighbours (K). An algorithm can have number of parameters and moreover there can be multiple parameters for each step of machine learning pipeline. There can be multiple combinations corresponding to possible values of all these parameters. We need parameter tunning to find such a combination of these parameters such that performance of our algorithm becomes maximum. In below attempts with different algorithms, we use GridSearch algorithm for tuning of parameters i.e. finding parameters which maximize specified performance score metric of algorithm.

In [10]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [11]:
# Learning code by grid search
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from tester import test_classifier

scaler = MinMaxScaler()
kbest = SelectKBest()

# first we try with decision tree
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier(random_state=42)
pipe = Pipeline([('kbest', kbest), ('clf_dt', clf_dt)])
parameters = {'kbest__k':range(5,15), 'clf_dt__max_depth':range(2,6), 'clf_dt__min_samples_leaf':range(1,5)}
folder = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
grid = GridSearchCV(pipe, param_grid=parameters, cv = folder, scoring = 'f1')
grid.fit(features, labels)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('kbest', SelectKBest(k=5, score_func=<function f_classif at 0x000000001951A438>)), ('clf_dt', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=3,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))])
	Accuracy: 0.82127	Precision: 0.28075	Recall: 0.21800	F1: 0.24543	F2: 0.22820
	Total predictions: 15000	True positives:  436	False positives: 1117	False negatives: 1564	True negatives: 11883



C:\Users\jmd\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


First Algorithm we tried above is Decision Tree classifier. As decision tree doesn't work on euclidean distances rather than depends on thresholds, so it doesn't require scaling. We also used SelectKBest module to find best features for this algorithm.  
In order to finetune our algorithm, we used following parameters in GridSearch, which runs the algorithm on all combinations of parameters and selects best scoring algorithm.
- k  
specifies number of features   

- max_depth  
sepcifies maximum depth of tree  

- min_samples_leaf  
Guarantees minimum number of samples in leaf of tree  

For validation of our algorithm, we used cross validation by which we split our data into training and testing set. By splitting our data, we won't overfit and could test on independent data set. Since our dataset is of small size(~140), we use stratified splits for number of iterations.  

Following metrics are used for validation
* ACCURACY  
This is just the ratio of correctly identified categories to total number of persons. In decision tree learner above, we have got high accuracy of 0.82, but it seems to be due to skewness in data as only few persons are of interest in total data. We see True Negative counts as staggering 11883 out of 15000 total predictions. To mitigate this bias, we use precision and recall defined below.  


* PRECISION  
Precision is ratio of correctly identified True positive to Total Positive predictions i.e.  
precision = True Positive(TP)/[True Positive(TP) + False Positive(FP)]  

In this experiment, precision refers to fraction of person of interest correctly identified out of total identified persons of interest. If we have low precision, we are incorrectly incriminating lots of false person of interest.  

We are getting moderate score of precision of 0.28 due to large numbers of False Positives (FP) with count of 1117. 

* RECALL  
Recall is measure of selecting maximum number of positive samples out of total positives. i.e.  
recall = True Positive(TP)/[True Positive(TP) + False Negative(FN)]  

In this experiment, recall refers to fraction of person of interest found out of total person of interests. If we have low recall, we are missing out lots of true person of interest.

We are getting very low quantity of recall (0.218) due to high numbers of false negatives (1564).

We would try other algoritm K-Nearest Neighbour(KNN) to improve our score.


In [12]:
# K-NN Learner
from sklearn.neighbors import KNeighborsClassifier
scaler = MinMaxScaler()
kbest = SelectKBest()
clf_knn = KNeighborsClassifier()
pipe = Pipeline([('scaler', scaler), ('kbest', kbest), ('clf_knn', clf_knn)])
parameters = {'kbest__k':range(5,15), 'clf_knn__n_neighbors':range(3,10)}
folder = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
grid = GridSearchCV(pipe, param_grid=parameters, cv = folder, scoring = 'f1')
grid.fit(features, labels)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('kbest', SelectKBest(k=5, score_func=<function f_classif at 0x000000001951A438>)), ('clf_knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'))])
	Accuracy: 0.82940	Precision: 0.26050	Recall: 0.15200	F1: 0.19198	F2: 0.16581
	Total predictions: 15000	True positives:  304	False positives:  863	False negatives: 1696	True negatives: 12137



As nearest neighbour algorithm is distance dependent, so we have added a scaler function in pipeline above, which would standardize the feature range. For tuning the algorithm, we use number of neighbours as a parameter.  
Following the description of evaluation metrics in Decision Tree, we see that precision and recall of KNN classifier are 0.26 and 0.15 respectively. These are very low performance, so we would try another algorithm Gaussian Naive Bayes in next section.

In [13]:
# Gaussian Naive Bayes Learner
from sklearn.naive_bayes import GaussianNB

kbest = SelectKBest()
clf_nb = GaussianNB()
pipe = Pipeline([('kbest', kbest), ('clf_nb', clf_nb)])
parameters = {'kbest__k':range(5,15)}
folder = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
grid = GridSearchCV(pipe, param_grid=parameters, cv = folder, scoring = 'f1')
grid.fit(features, labels)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('kbest', SelectKBest(k=6, score_func=<function f_classif at 0x000000001951A438>)), ('clf_nb', GaussianNB())])
	Accuracy: 0.85073	Precision: 0.42788	Recall: 0.35450	F1: 0.38775	F2: 0.36709
	Total predictions: 15000	True positives:  709	False positives:  948	False negatives: 1291	True negatives: 12052



In above Naive bayes classifier, we see comparitively high precision(0.42) and recall (0.35), both greater that 0.3. So we choose it as our final model.  
As Naive Bayes doesn't depends upon distances, so no feature scaling is required.  



In [14]:
print "scores with different values of K i.e numbers of features"
grid.grid_scores_

scores with different values of K i.e numbers of features


[mean: 0.39771, std: 0.31880, params: {'kbest__k': 5},
 mean: 0.40871, std: 0.32005, params: {'kbest__k': 6},
 mean: 0.37871, std: 0.31796, params: {'kbest__k': 7},
 mean: 0.34971, std: 0.32040, params: {'kbest__k': 8},
 mean: 0.34100, std: 0.32100, params: {'kbest__k': 9},
 mean: 0.34121, std: 0.30976, params: {'kbest__k': 10},
 mean: 0.34006, std: 0.29647, params: {'kbest__k': 11},
 mean: 0.31428, std: 0.29492, params: {'kbest__k': 12},
 mean: 0.30488, std: 0.29482, params: {'kbest__k': 13},
 mean: 0.28962, std: 0.27850, params: {'kbest__k': 14}]

We can see in above summay that score is highest corresponding to 6 feature selected by k-best.

As selector is being passed in grid estimator, we explicitly find approximate features for k=6. In below results, we see that newly added feature 'fraction_to_poi' is one of feature selected for predicting final outcome.

In [23]:
kbest_exp = SelectKBest(k=6)
kbest_exp.fit(features, labels)
features_selected = [features_list[i+1] for i in kbest_exp.get_support(indices=True)]
print 'features selected by kbest'
print features_selected

features selected by kbest
['salary', 'bonus', 'total_stock_value', 'exercised_stock_options', 'deferred_income', 'fraction_to_poi']


To find effect of our new feature, we find best score without our new features

In [24]:
# remove 2 new features
features_list.remove("fraction_from_poi")
features_list.remove("fraction_to_poi")

from sklearn.naive_bayes import GaussianNB

kbest = SelectKBest()
clf_nb = GaussianNB()
pipe = Pipeline([('kbest', kbest), ('clf_nb', clf_nb)])
parameters = {'kbest__k':range(5,15)}
folder = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
grid = GridSearchCV(pipe, param_grid=parameters, cv = folder, scoring = 'f1')
grid.fit(features, labels)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('kbest', SelectKBest(k=6, score_func=<function f_classif at 0x000000001951A438>)), ('clf_nb', GaussianNB())])
	Accuracy: 0.84600	Precision: 0.40264	Recall: 0.32050	F1: 0.35690	F2: 0.33413
	Total predictions: 15000	True positives:  641	False positives:  951	False negatives: 1359	True negatives: 12049



We can see that Precision score is decreased to 0.40 from 0.42 and Recall is decreases to 0.32 from 0.35 without our new features confirming usefulness of our new feature.